# Observable and Pauli String representation

This notebook provides an exploration of quantum circuit observable representation and the creation and manipulation of PauliString objects.
### Overview :
- **PauliString**: This class allows for the creation, manipulation, and arithmetic operations on Pauli strings.
- **Observables**: You can define an `Observable` using Hermitian matrices or Pauli strings. Both approaches provide flexibility in specifying observables for quantum measurements.
- **ExpectationMeasure**: Create and run quantum circuits with `ExpectationMeasure` to calculate expectation values of observables.

In [1]:
from mpqp.measures import I, X, Y, Z, Observable, ExpectationMeasure

from mpqp import QCircuit
from mpqp.gates import H, Rx
from mpqp.execution import run
from mpqp.execution.devices import IBMDevice, ATOSDevice, AWSDevice

import numpy as np

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


> ⚠ **pauli_string import**: pauli atoms are named I, X, Y, and Z. If you have conflicts with `mpqp.gates import X, Y, Z,` , you can:
> - **Rename Import:**
>    ```python 
>    from mpqp.core.instruction.measurement.pauli_string import X as Pauli_X 
>    ```
> - **Import Only Pauli String:**
>    ```python
>    from mpqp.core.instruction.measurement import pauli_string 
>    pauli_string.X 
>    ```

## PauliString

A `PauliString` is based on the following hierarchy:
- an *atom* is the most elemental building brick of pauli string, it it either
  `I`, `X`, `Y` or `Z`;
- a *monomial* is a tensor product of *atoms* multiplied by a real coefficient,
  for instance `0.3 * I⊗Z⊗Y`. In MPQP, the tensor product is denoted as `@`, so
  the previous example would be expressed as `0.3 * I@Z@Y`;
- a *string* is a sum of *monomials*, for instance `0.3 * I@Z@Y + X@X@X`.

In practice, you never need to handle these types independently, you can express
everything in term of expression based on the fundamental atoms. Let's see how.

### Creating and Manipulating PauliString Objects

Let's illustrate how to create and manipulate PauliString objects:

In [2]:
ps_1 = I @ Z - 3 * X @ Y
print(f"{ps_1=}")

ps_1=1*I@Z + -3*X@Y


#### Simplifying and Rounding PauliStrings

When dealing with PauliStrings, simplification and rounding are common operations:

- **Simplify**: We combine like terms and eliminate those with zero coefficients;
- **Round**: We can round the coefficients to a specified number of decimals (5 by default).

> `str` on a `PauliString` will call both methods: `round()` and `simplify()`

In [3]:
ps_2 = I @ Z + 2.555555555 * Y @ I + X @ Z - X @ Z
print("ps_2 =",repr(ps_2))
print("     =",repr(ps_2.simplify()))
print("    ~=",repr(ps_2.round(1)))
print("    ~=",ps_2)

ps_2 = 1*I@Z + 2.555555555*Y@I + 1*X@Z + -1*X@Z
     = 1*I@Z + 2.555555555*Y@I
    ~= 1*I@Z + 2.6*Y@I + 1*X@Z + -1*X@Z
    ~= 1*I@Z + 2.5556*Y@I


#### Arithmetic Operations

We can perform various arithmetic operations on PauliString objects, including addition, subtraction, scalar multiplication, scalar division, and matrix multiplication:

In [5]:
ps_2 = ps_2.round(1).simplify()
print(f"""Addition:
({ps_1}) + ({ps_2}) = {ps_1 + ps_2}

Subtraction:
({ps_1}) - ({ps_2}) = {ps_1 - ps_2}

Scalar product:
2 * ({ps_1}) = {2 * ps_1}

Scalar division:
({ps_2}) / 3 ~= {ps_2 / 3}

Tensor product:
({ps_1}) @ Z = {ps_1 @ Z}

({ps_1}) @ ({ps_2}) = {ps_1 @ ps_2}""")

Addition:
(1*I@Z + -3*X@Y) + (1*I@Z + 2.6*Y@I) = 2*I@Z + 2.6*Y@I + -3*X@Y

Subtraction:
(1*I@Z + -3*X@Y) - (1*I@Z + 2.6*Y@I) = -2.6*Y@I + -3*X@Y

Scalar product:
2 * (1*I@Z + -3*X@Y) = 2*I@Z + -6*X@Y

Scalar division:
(1*I@Z + 2.6*Y@I) / 3 ~= 0.3333*I@Z + 0.8667*Y@I

Tensor product:
(1*I@Z + -3*X@Y) @ Z = -3*X@Y@Z + 1*I@Z@Z

(1*I@Z + -3*X@Y) @ (1*I@Z + 2.6*Y@I) = 1*I@Z@I@Z + -3*I@Z@X@Y + -7.8*Y@I@X@Y + 2.6*Y@I@I@Z


## Observable

To compute the expectation value of the state generated by a circuit measured by
an observable, we first define the observable. 

It can be instantiated either using a matrix (Hermitian) or a Pauli string.

In [12]:
matrix = np.array(
    [
        [0.65, 0.5, 1, 1],
        [0.5, 0.82, 1, 1],
        [1, 1, 1, 0.33],
        [1, 1, 0.33, 0.3],
    ]
)

obs = Observable(matrix)

print("`obs` was created from the matrix:")
print(obs.matrix)

obs2 = Observable(ps_1)

print("\n`obs2` was created from the Pauli string:")
print(obs2.pauli_string)

`obs` was created from the matrix:
[[0.65+0.j 0.5 +0.j 1.  +0.j 1.  +0.j]
 [0.5 +0.j 0.82+0.j 1.  +0.j 1.  +0.j]
 [1.  +0.j 1.  +0.j 1.  +0.j 0.33+0.j]
 [1.  +0.j 1.  +0.j 0.33+0.j 0.3 +0.j]]

`obs2` was created from the Pauli string:
1*I@Z + -3*X@Y


Since there is an equivalence between definition from a Pauli string or from a
Hermitian matrix, both these observable wan can also be expressed in term of the
mean through which it was not defined (Pauli for matrix and vice versa):

In [13]:
print("`obs` as a Pauli string:")
print(obs.pauli_string)

print("\n`obs2` as a matrix:")
print(obs2.matrix)

`obs` as a Pauli string:
0.0425*Z@I + -0.2175*Z@Z + 0.085*Z@X + 0.6925*I@I + 1*X@I + 0.1325*I@Z + 0.415*I@X + 1*X@X

`obs2` as a matrix:
[[ 1.+0.j  0.+0.j  0.+0.j  0.-3.j]
 [ 0.+0.j -1.+0.j  0.+3.j  0.+0.j]
 [ 0.+0.j  0.-3.j  1.+0.j  0.+0.j]
 [ 0.+3.j  0.+0.j  0.+0.j -1.+0.j]]


## ExpectationMeasure

Next, we create a quantum circuit to measure the expectation value of the observable. We add gates and the ExpectationMeasure to the circuit.

In [8]:
circuit = QCircuit([
    H(0), 
    Rx(1.76, 1), 
    ExpectationMeasure([0, 1], observable=obs, shots=1000),
])

### Running the Circuit and Retrieving Results

We run the circuit on different quantum simulators and retrieve the results.

In [14]:
results = run(
    circuit,
    [
        ATOSDevice.MYQLM_PYLINALG,
        IBMDevice.AER_SIMULATOR,
        ATOSDevice.MYQLM_CLINALG,
        AWSDevice.BRAKET_LOCAL_SIMULATOR,
    ],
)
print(results)

This program uses OpenQASM language features that may not be supported on QPUs or on-demand simulators.



BatchResult: 4 results
Result: IBMDevice, AER_SIMULATOR
 Expectation value: 1.7161599964797496
 Error/Variance: 1.2440903362350237
Result: ATOSDevice, MYQLM_CLINALG
 Expectation value: 1.6611499969959258
 Error/Variance: 0.035304078895363314
Result: AWSDevice, BRAKET_LOCAL_SIMULATOR
 Expectation value: 1.7169993078883168
 Error/Variance: None
Result: ATOSDevice, MYQLM_PYLINALG
 Expectation value: 1.6149999956786634
 Error/Variance: 0.03527477642013116

